#UTILS.PY

In [ ]:
!nvidia-smi

Wed Aug 26 08:14:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install segmentation_models_pytorch

  Using cached https://files.pythonhosted.org/packages/70/88/763a25dfe076a9f30f33466b1bd0f2d31b915b88d4cb4481fe4043cf26b4/segmentation_models_pytorch-0.1.0-py3-none-any.whl
Processing /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6/efficientnet_pytorch-0.7.0-cp36-none-any.whl
Processing /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd/pretrainedmodels-0.7.4-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/ab/85d8da5c9a45e072301beb37ad7f833cd344e04c817d97e0cc75681d248f/munch-2.5.0-py2.py3-none-any.whl


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd


def mask_to_rle(mask):
    """
    mask:  numpy array,  1 - mask, 0 - background
    return: run length as string formatted
    """
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[:-1] != pixels[1:])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_to_mask(rle, shape=(1600, 256)):
    """
    :param rle: run-length as string formated (start length)
    :param shape: (width,height) of array to return
    :return: numpy array, 1 - mask, 0 - background
    """
    runs = np.array([int(x) for x in rle.split()])
    runs[1::2] += runs[::2]
    runs -= 1
    starts, ends = runs[::2], runs[1::2]
    mask = np.zeros(shape[0] * shape[1])
    for start, end in zip(starts, ends):
        mask[start:end] = 1
    return mask.reshape(shape).T


def show_defects(image, mask, pallet=((249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12))):

    for i in range(4):
        image[0, mask[i] == 1] = 255
    plt.imshow(image.permute(1, 2, 0))
    plt.show()


def show_mask_image(image, mask, pallet=((249, 192, 12), (0, 185, 241), (114, 0, 218), (249, 50, 12))):
    fig, ax = plt.subplots(figsize=(15, 15))
    image = image.permute(1, 2, 0).numpy()
    mask = mask.permute(1, 2, 0)

    for ch in range(4):
        image[mask[:, :, ch] == 1] = pallet[ch]
    plt.imshow(image)

    plt.show()


def make_mask(name, df):
    mask = np.zeros((256, 1600, 4), dtype=np.float32)
    rows = df.loc[name]
    for defect in range(1, 5):
        rle = rows[defect]
        if not pd.isna(rle):
            encoded = rle_to_mask(rle)
            mask[:, :, defect - 1] = encoded
    return mask

import torch


def dice_single_channel(targets, preds, eps=1e-9):
    batch_size = preds.shape[0]
    preds = preds.view((batch_size, -1)).float()
    targets = targets.view((batch_size, -1)).float()
    dice = (2 * (preds * targets).sum(1) + eps) / (preds.sum(1) + targets.sum(1) + eps)
    return dice


def mean_dice_score(targets, outputs, threshold=0.5):
    batch_size = outputs.shape[0]
    n_channels = outputs.shape[1]
    preds = (outputs.sigmoid() > threshold).float()

    mean_dice = 0
    for i in range(n_channels):
        dice = dice_single_channel(targets[:, i, :, :], preds[:, i, :, :])
        mean_dice += dice.sum(0) / (n_channels * batch_size)
    return mean_dice.item()


def pixel_accuracy_score(targets, outputs, threshold=0.5):
    preds = (outputs.sigmoid() > threshold).float()
    correct = torch.sum((targets == preds)).item()
    total = outputs.numel()
    return correct / total


def epoch_metrics(targets, outputs, threshold=0.5):
    return {'dice': mean_dice_score(targets, outputs, threshold),
            'pixel_acc': pixel_accuracy_score(targets, outputs, threshold)}


def predict(output, threshold=0.5):
    prediction = (output.sigmoid() > threshold).float()
    return prediction

from torch.utils.data import DataLoader, Dataset
import pandas as pd
import cv2
import os
import albumentations as albu
import albumentations.pytorch as albu_pytorch
from sklearn.model_selection import train_test_split


class SteelDataset(Dataset):
    def __init__(self, dataset, phase='train', data_dir='train_images', image_size=(256, 1600), n_classes=4):
        self.dataset = dataset
        self.phase = phase
        self.dir = data_dir
        self.transforms = get_transforms(phase=self.phase)
        self.image_size = image_size
        self.n_classes = n_classes

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        name = self.dataset.iloc[index].name
        image = cv2.imread(os.path.join(self.dir, name))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = make_mask(name, self.dataset)

        transformed = self.transforms(image=image, mask=mask)
        image, mask = transformed['image'], transformed['mask'][0].permute(2, 0 , 1)

        return image, mask


def get_transforms(list_transforms=None, phase='train'):
    if not list_transforms:
        list_transforms = []

    if phase == 'train':
        list_transforms.extend(
            [
                albu.RandomBrightnessContrast(p=0.1, brightness_limit=0.1, contrast_limit=0.1),
                albu.HorizontalFlip(p=0.65),
                albu.VerticalFlip(p=0.65),
                # albu.ElasticTransform(p=0.5),
                # albu.GridDistortion(p=0.5),
                # albu.OpticalDistortion(p=0.5),
            ]
        )
    list_transforms.extend(
        [
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            albu_pytorch.ToTensor()
        ]
    )

    list_transforms = albu.Compose(list_transforms)
    return list_transforms


def data_provider(df, batch_size=8, shuffle=True, stratify_by=None):

    if stratify_by:
        train_df, val_df = train_test_split(df, test_size=0.2,
                                            stratify=df[stratify_by],
                                            random_state=42,
                                            shuffle=shuffle)
    else:
        train_df, val_df = train_test_split(df, test_size=0.2,
                                            random_state=42,
                                            shuffle=shuffle)

    dataloader = {'train': DataLoader(SteelDataset(train_df, phase='train'), batch_size=batch_size),
                  'val': DataLoader(SteelDataset(val_df, phase='val'), batch_size=batch_size)}


    return dataloader




# Radam

In [ ]:
from torch.optim.optimizer import Optimizer
import math
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-5, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt(
                            (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (
                                    N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-1 * step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-1 * step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss



# Loading data

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

files = ['/content/drive/My Drive/severstal/steel.zip', '/content/drive/My Drive/severstal/test_images.zip']
for path_zip_file in files:
    z = zipfile.ZipFile(path_zip_file, 'r')
    z.extractall()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# MODEL.PY

In [ ]:
import torch
from torch import nn
import segmentation_models_pytorch as smp
import numpy as np
import pandas as pd
import time
from tqdm import tqdm_notebook as tqdm

DIR_TO_SAVE_MODELS = '/content/drive/My Drive/severstal/models'
DIR_TO_SAVE_VALUES = '/content/drive/My Drive/severstal/values'
DIR_TO_SAVE_LOG = '/content/drive/My Drive/severstal/log'

class Trainer:
    def __init__(self, model, criterion, optimizer, scheduler, device, data_frame, batch_size=8, stratify_by=None):
        self.model = model
        self.device = device
        self.model = model.to(device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.batch_size = batch_size
        self.accumulation_steps = 256 // self.batch_size
        self.df = data_frame
        self.dataloaders = data_provider(self.df, batch_size=self.batch_size, stratify_by=stratify_by)
        self.losses = {phase: [] for phase in ['train', 'val']}
        self.metrics = {'dice': mean_dice_score, 'pixel_acc': pixel_accuracy_score}
        self.metrics_values = {phase: {name: [] for name in self.metrics.keys()}
                               for phase in ['train', 'val']}
        self.best_score = np.array([-np.inf for _ in self.metrics.keys()])

    def step(self, epoch, phase):

        epoch_loss = 0.0
        metrics = {name: [] for name in self.metrics.keys()}
        epoch_metric = {}

        if phase == 'train':
            self.model.train()
        else:
            self.model.eval()

        dataloader = self.dataloaders[phase]
        pbar = tqdm(dataloader, total=len(dataloader))

        self.optimizer.zero_grad()
        for i, (images, targets) in enumerate(dataloader):
            images, targets = images.to(self.device), targets.to(self.device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)

                if phase == "train":
                    loss.backward()
                    if (i + 1) % self.accumulation_steps == 0:
                        self.optimizer.step()
                        self.optimizer.zero_grad()

                for metric in self.metrics.keys():
                    metrics[metric].append(self.metrics[metric](targets, outputs))

                epoch_loss += loss.item()
                pbar.update(1)
                
        pbar.close()
        epoch_loss = (epoch_loss * self.accumulation_steps) / len(dataloader)
        self.losses[phase].append(epoch_loss)

        for metric in self.metrics.keys():
            epoch_metric[metric] = np.mean(metrics[metric])
            self.metrics_values[phase][metric].append(epoch_metric[metric])

        del images, targets, outputs, loss
        torch.cuda.empty_cache()
        return epoch_loss, epoch_metric

    def train(self, num_epochs):
        for epoch in tqdm(range(num_epochs)):
            loss, metric = self.step(epoch, 'train')
            print('Epoch {} | train_loss {} | train_metric {}'.format(epoch, loss, metric))
            state = {'epoch': epoch,
                     'best_score': self.best_score,
                     'state_dict': self.model.state_dict(),
                     'optimizer': self.optimizer.state_dict()}

            loss, metric = self.step(epoch, 'val')
            print('Epoch {} | val_loss {} | val_metric {}'.format(epoch, loss, metric))
            self.scheduler.step(loss)
            scores = np.fromiter(metric.values(), dtype=np.float)
            if (scores[0] > self.best_score[0]).all():
                print('-' * 10 + 'New optimal model found and saved' + '-' * 10)
                state['best_metric'] = metric
                torch.save(state, "{}/model_epoch_{}_score_{:.4f}.pth".format(DIR_TO_SAVE_MODELS, epoch, scores[0]))
     
                losses_file = open("{}/losses/loss_epoch_{}.json".format(DIR_TO_SAVE_VALUES, epoch), "w")
                json.dump(self.losses, losses_file)
                losses_file.close() 

                metrics_file = open("{}/metric/metric_epoch_{}.json".format(DIR_TO_SAVE_VALUES, epoch), "w")
                json.dump(self.metrics_values, metrics_file)
                metrics_file.close() 
    
                self.best_score = scores

            losses_file = open("{}/losses/loss_epoch_{}.json".format(DIR_TO_SAVE_LOG, epoch), "w")
            json.dump(self.losses, losses_file)
            losses_file.close() 

            metrics_file = open("{}/metric/metric_epoch_{}.json".format(DIR_TO_SAVE_LOG, epoch), "w")
            json.dump(self.metrics_values, metrics_file)
            metrics_file.close()
    

In [ ]:
import torch.nn.functional as F

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = 0.35 * BCE + 0.65 * dice_loss
        
        return Dice_BCE

In [ ]:
import json
if __name__ == '__main__':
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = smp.Unet('resnet34', encoder_weights=None, classes=4, activation=None)
    state = torch.load('/content/drive/My Drive/severstal/models/model_epoch_23_score_0.8903.pth')
    model.load_state_dict(state['state_dict'])
    criterion = DiceBCELoss()
    optimizer = RAdam(model.parameters(), lr=5e-4)
    # optimizer = torch.optim.Adam(model.parameters(), lr=5e-6)
    optimizer.load_state_dict(state['optimizer'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, mode="min", patience=3, verbose=True)

    df = pd.read_csv('/content/train.csv')
    df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
    df['NumDefects'] = df.count(axis=1)

    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    model_train = Trainer(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        batch_size=8,
        data_frame=df,
        stratify_by='NumDefects'
    )

model_train.best_score = [0.8903, -np.inf]

In [ ]:
model_train.optimizer

RAdam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.00014348907
    weight_decay: 0
)

In [ ]:
 for param_group in model_train.optimizer.param_groups:
        param_group['lr'] = 3e-4

In [ ]:
path = '/content/drive/My Drive/severstal/log/losses'
file_list = os.listdir(path)
full_list = [os.path.join(path, i) for i in file_list]
time_sorted_list = sorted(full_list, key=os.path.getmtime, reverse=True)
last_log = time_sorted_list[0]

with open(os.path.join(path, last_log)) as json_file:
    loss = json.load(json_file)

model_train.losses['train'] = loss['train']
model_train.losses['val'] = loss['val']

In [ ]:
path = '/content/drive/My Drive/severstal/log/metric'
file_list = os.listdir(path)
full_list = [os.path.join(path, i) for i in file_list]
time_sorted_list = sorted(full_list, key=os.path.getmtime, reverse=True)
last_log = time_sorted_list[0]

with open(os.path.join(path, last_log)) as json_file:
    metric = json.load(json_file)

model_train.metrics_values['train']['dice'] = metric['train']['dice']
model_train.metrics_values['val']['dice'] = metric['val']['dice']
model_train.metrics_values['train']['pixel_acc'] = metric['train']['pixel_acc'] 
model_train.metrics_values['val']['pixel_acc']  = metric['val']['pixel_acc'] 

In [ ]:
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
model_train.train(20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Epoch 0 | train_loss 1.8908817718054043 | train_metric {'dice': 0.952299028322257, 'pixel_acc': 0.9977469067666486}



Epoch 0 | val_loss 5.677879433432025 | val_metric {'dice': 0.8898119944298338, 'pixel_acc': 0.9934844876287462}


...


Epoch 10 | train_loss 1.9543788762821788 | train_metric {'dice': 0.9464657807993567, 'pixel_acc': 0.9976520094521222}



Epoch 10 | val_loss 5.659140237077268 | val_metric {'dice': 0.8900469709299281, 'pixel_acc': 0.9932442101271093}



Epoch 11 | train_loss 1.944914953819458 | train_metric {'dice': 0.9452934154327484, 'pixel_acc': 0.9976644976385709}



Epoch 11 | val_loss 5.6237357248089275 | val_metric {'dice': 0.8895739666716067, 'pixel_acc': 0.993281868118012}



Epoch 12 | train_loss 1.9552524849392663 | train_metric {'dice': 0.9439882833560903, 'pixel_acc': 0.9976455734230052}


# submitions

In [ ]:
import pandas as pd
import torch
import os
import cv2
import segmentation_models_pytorch as smp
from tqdm import tqdm_notebook as tqdm

def test_generator(transforms, path_to_images='test_images'):
    images_name = os.listdir(path_to_images)

    for name in images_name:
        image = cv2.imread(os.path.join(path_to_images, name))
        image = transforms(image=image)['image']
        yield image, name


def make_submission(
        model,
        path_to_images='test_images',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')):

    model.to(device)
    model.eval()
    transforms = get_transforms(phase='val')
    test_images = test_generator(transforms, path_to_images)
    result = pd.DataFrame(columns=['ImageId', 'EncodedPixels', 'ClassId'])
    pbar = tqdm(test_images, total=5506)
    for image, name in test_images:
        output = predict(model(image.unsqueeze(0).to(device)).cpu().detach())
        for defect in range(4):
            rle = mask_to_rle(output[:, defect, :, :])
            result = result.append({'ImageId': name, 'EncodedPixels': rle, 'ClassId': defect + 1}, ignore_index=True)
        pbar.update(1)
    pbar.close()
    return result


if __name__ == '__main__':
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = smp.Unet('resnet18', classes=4, activation=None)
    model.load_state_dict(torch.load('/content/drive/My Drive/models/model_epoch_19_score_0.8847.pth', map_location=device)['state_dict'])
    result = make_submission(model, path_to_images='/content/test_images')
    result.to_csv("submission.csv", index=False)
    print(result)